# Extracting Structured Data from Images
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Structured_Text_Extraction_from_Images.ipynb)

## Introduction

In this notebook we will demonstrate how you can use a language vision model(Llama 3.2 90B Vision) along with an LLM that has JSON mode enabled(Llama 3.1 70B) to extract structured text from images.

In our case we will extract line items from an invoice in the form of a JSON.

<img src="images\structured_text_image.png" width="750">


### Install relevant libraries

In [1]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 1.9 MB/s eta 0:00:00


In [4]:
import together, os

# Paste in your Together AI API Key or load it
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

## Create Invoice Structure using Pydantic

We need a way of telling the LLM what structure to organize information into - including what information to expect in the receipt. We will do this using `pydantic` models.

Below we define the required classes.

- Each line item on the receipt will have a `name`, `price` and `quantity`. The `Item` class specifies this.
- Each receipt/invoice is a combination of multiple line `Item` elements along with a `total` price. The `Receipt` class specifies this.

In [12]:
import json
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str
    price: float
    quantity: int = Field(default=1)

class Receipt(BaseModel):
    items: list[Item]
    total: float

## Lets bring in the reciept that we want to extract information from

Notice that this is a real receipt with multiple portions that are not relevant to the line item extraction structure we've outlined above.

<img src="https://ocr.space/Content/Images/receipt-ocr-original.webp" height="500">

## 1. Extract Information Receipt

We will use the Llama 3.2 90B Vision model to extract out information in normal text format.

In [16]:
from together import Together

getDescriptionPrompt = "Extract out the details from each line item on the receipt image. Identify the name, price and quantity of each item. Also specify the total."

imageUrl = "https://ocr.space/Content/Images/receipt-ocr-original.webp"

client = Together(api_key=TOGETHER_API_KEY)

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": getDescriptionPrompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl,
                    },
                },
            ],
        }
    ],
)

info = response.choices[0].message.content

In [17]:
print(info)

The receipt shows a total of 17 line items. The details for each item are as follows:

1. Pet Toy: $1.97, quantity - 1
2. Floppy Puppy: $1.97, quantity - 1
3. Sssupreme S: $4.97, quantity - 1
4. 2.5 Squeak: $5.92, quantity - 1
5. Munchy Dmbel: $3.77, quantity - 1
6. Dog Treat: $2.92, quantity - 1
7. Ped Pch 1: $0.50, quantity - 1 (x2)
8. Coupon: $1.00, quantity - 1
9. Hnymd Smores: $3.98, quantity - 1
10. French Drsng: $1.98, quantity - 1
11. 3 Oranges: $5.47, quantity - 1
12. Baby Carrots: $1.48, quantity - 1
13. Collards: $1.24, quantity - 1
14. Calzone: $2.50, quantity - 1
15. Mm Rvw Mnt: $19.77, quantity - 1
16. Stkobrlplabl: $1.97, quantity - 1 (x6)
17. Dry Dog: $12.44, quantity - 1

The total is $98.21.


Notice that the model is not perfect and wasn't able to extract out some line items. It's hard for most models to perform this zero-shot extraction of data from images. A way to improve this is to finetune the model using [Visual Intruction Tuning](https://arxiv.org/abs/2304.08485).

## 2. Organize Information as JSON

We will use Llama 3.1 70B with structured generation in JSON mode to organize the information extracted by the vision model into an acceptable JSON format that can be parsed.

`Meta-Llama-3.1-70B-Instruct-Turbo` will strcitly respect the JSON schema passed to it.

In [18]:
extract = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "The following is a detailed description of all the items, prices and quantities on a receipt. Extract out information. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": info,
            },
        ],
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": Receipt.model_json_schema(),
        },
    )

In [19]:
output = json.loads(extract.choices[0].message.content)
print(json.dumps(output, indent=2))

{
  "items": [
    {
      "name": "Pet Toy",
      "price": 1.97,
      "quantity": 1
    },
    {
      "name": "Floppy Puppy",
      "price": 1.97,
      "quantity": 1
    },
    {
      "name": "Sssupreme S",
      "price": 4.97,
      "quantity": 1
    },
    {
      "name": "2.5 Squeak",
      "price": 5.92,
      "quantity": 1
    },
    {
      "name": "Munchy Dmbel",
      "price": 3.77,
      "quantity": 1
    },
    {
      "name": "Dog Treat",
      "price": 2.92,
      "quantity": 1
    },
    {
      "name": "Ped Pch 1",
      "price": 0.5,
      "quantity": 2
    },
    {
      "name": "Coupon",
      "price": -1.0,
      "quantity": 1
    },
    {
      "name": "Hnymd Smores",
      "price": 3.98,
      "quantity": 1
    },
    {
      "name": "French Drsng",
      "price": 1.98,
      "quantity": 1
    },
    {
      "name": "3 Oranges",
      "price": 5.47,
      "quantity": 1
    },
    {
      "name": "Baby Carrots",
      "price": 1.48,
      "quantity": 1
    },
 

Althought with some missed line items we were able to extract out structured JSON from an image in a zero shot manner! To improve the results for your pipeline and make them production ready I recommend you [finetune](https://docs.together.ai/docs/fine-tuning-overview) the vision model on your own dataset!

Learn more about how to use JSON mode in the [docs](https://docs.together.ai/docs/json-mode) here!